<a href="https://colab.research.google.com/github/limesun/Transfer-Learning/blob/master/ULMFiT_20_newsgroup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai 

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
    100% |████████████████████████████████| 576.2MB 27kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x62ebe000 @  0x7f6370e452a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
  Running setup.py bdist_wheel for wrapt ... - \ | done
  Stored in directory: /root/.cache/pip/wheels/48/5d/04/22361a593e70d23b1f7746d932802efe1f0e523376a74f321e
Successfully built wrapt
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
  Found existing installation: wrapt 1.11.1
    Uninstalling wrapt-1.11.1:
      Successfully uninstalled wrapt-1.11.1


In [0]:
# import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [0]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data


In [0]:
documents[:3]

["Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n",
 "\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap

In [0]:
dataset.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [0]:
df = pd.DataFrame({'label':dataset.target, 'text':dataset.data})

In [0]:
df.shape

(11314, 2)

In [0]:
df.head()

,label,text
0,17,Well i'm not sure about the story nad it did s...
1,0,"\n\n\n\n\n\n\nYeah, do you expect people to re..."
2,17,Although I realize that principle is not one o...
3,11,Notwithstanding all the legitimate fuss about ...
4,10,"Well, I will have to change the scoring on my ..."


In [0]:
df.label[df.label == 8].index.tolist()

In [0]:
df.groupby(['label']).count()

,text
label,
0,480
1,584
2,591
3,590
4,578
5,593
6,585
7,594
8,598


**Model performance test on 20newsgroup dataset by making sub-datasets with 2 categories**

In [0]:
df1 = df[df['label'].isin([1,10])]  #  1 = comp.graphic, 10 = rec.sport.hockey
df1 = df1.reset_index(drop = True)

In [0]:
df1.head()

,label,text
0,10,"Well, I will have to change the scoring on my ..."
1,1,Archive-name: graphics/resources-list/part1\nL...
2,10,"\nAnd of course, Mike Ramsey was (at one time)..."
3,10,"As I promised, I would give you the name of th..."
4,10,GAME(S) OF 4/15\n---------------\nADIRONDACK 6...


In [0]:
df1['label'].value_counts()

10    600
1     584
Name: label, dtype: int64

In [0]:
dic = dict (enumerate(df1.label.unique()))
dic


{0: 10, 1: 1}

In [0]:
df1[df1['label'] == 10]

,label,text
0,10,"Well, I will have to change the scoring on my ..."
2,10,"\nAnd of course, Mike Ramsey was (at one time)..."
3,10,"As I promised, I would give you the name of th..."
4,10,GAME(S) OF 4/15\n---------------\nADIRONDACK 6...
6,10,"You think that's bad? I'm in Bowling Green, O..."
7,10,] \tAnyone who really believes that the Caps c...
8,10,^^^^^^^^^^^^^^^^^^^^^^^^\n ...
9,10,\nChris Chelios was Montreal's co-captain with...
12,10,Has anyone heard what game ESPN is showing ton...
13,10,


In [0]:
#Data Preprocessing 
df1['text'] = df1['text'].str.replace("[^a-zA-Z]", " ")

In [0]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
# tokenization 
tokenized_doc = df1['text'].apply(lambda x: x.split())

# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# de-tokenization 
detokenized_doc = [] 
for i in range(len(df1)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

df1['text'] = detokenized_doc

In [0]:
from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn, df_val = train_test_split(df1, stratify = df1['label'], test_size = 0.4, random_state = 12)

In [0]:
df_trn.shape, df_val.shape

((710, 2), (474, 2))

In [0]:
df_val.head()

,label,text
490,1,I looking source code radiosity method I know ...
1102,1,I interested information stereoscopic imaging ...
557,10,Does anyone NHL STANDINGS March th I need IMME...
209,10,A little supplement Basic Mike info For Sundda...
774,10,World Championships Germany Group A results SW...


In [0]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

In [0]:
#Fine-Tuning 
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103, drop_mult=0.7)


In [0]:
# train the learner object with learning rate = 1e-2
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,7.918004,6.472476,0.126269


In [0]:
learn.save_encoder('ft_enc')

In [0]:
learn = text_classifier_learner(data_clas, drop_mult=0.7)
learn.load_encoder('ft_enc')

In [0]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.523875,0.326793,0.919831


In [0]:
# get predictions - {0: 10, 1: 1}
preds, targets = learn.get_preds()

predictions = np.argmax(preds, axis = 1)
pd.crosstab(predictions, targets)

col_0,0,1
row_0,,
0,199,3
1,35,237


In [0]:
targets   # 0 = label 1 / 10 = label 10

tensor([0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
        1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
        0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0,
        1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
        0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
        1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,

In [0]:
df_val.head()

,label,text
412,3,Could one tell Phar Lap err The chip currently...
1158,4,You need add two K VRAM SIMMs K VRAM SIMMs wor...
210,3,I would doubt would able use hard drive XT dri...
568,4,
361,4,No problems


In [0]:

# Label eye checking results
# 순서대로 되어 있음

#16 = talk.politics.guns/ 14 = sci.space / 4 = comp.sys.mac.hardware / 1 = comp.graphic / 10 = rec.sport.hockey

**Model performance test on 20newsgroup dataset by making sub-datasets with 5 categories which is similar each other**

In [0]:

df2 = df[df['label'].isin([1, 2, 3, 4, 5])]  # 1=comp.graphic, 2= comp.os.ms-windows.misc, 3 = comp.sys.ibm.pc.hardware, 4 = comp.sys.mac.hardware, 5=comp.windows.x, 
df2 = df2.reset_index(drop = True)

In [0]:
df2.index.name = 'index'
#dic = {1:0, 2:1, 3:2, 4:3, 5:4}
#df2.index.name = dic

In [0]:
df2[:20]

,label,text
6,4,\nOk. I have a record that shows a IIsi with ...
10,1,Archive-name: graphics/resources-list/part1\nL...
14,4,"The title says it all. I need to know the 44,..."
31,4,\nI doubt this is a Quadra-specific problem. ...
32,4,"\n\nTerry, hi. I recently bought an LCIII and..."
38,4,\nBattery powered devices like the PowerBook a...
40,4,terminated\n\nIt is very possible to connect a...
43,1,"Hi,\n\nI'm interested in writing a program to ..."
44,2,.Chris\n
45,5,*Reminder* Plan now for the Andrew Conferenc...


In [0]:
df2['label'].value_counts()

5    593
2    591
3    590
1    584
4    578
Name: label, dtype: int64

In [0]:
#Data Preprocessing 
df2['text'] = df2['text'].str.replace("[^a-zA-Z]", " ")

In [0]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
# tokenization 
tokenized_doc = df2['text'].apply(lambda x: x.split())

# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# de-tokenization 
detokenized_doc = [] 
for i in range(len(df2)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

df2['text'] = detokenized_doc

In [0]:
from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn, df_val = train_test_split(df2, stratify = df2['label'], test_size = 0.3, random_state = 12)

In [0]:
df_trn.shape, df_val.shape

((2055, 2), (881, 2))

In [0]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

In [0]:
#Fine-Tuning 
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103, drop_mult=0.7)

In [0]:
data_clas

TextClasDataBunch;

Train: LabelList
y: CategoryList (2055 items)
[Category 2, Category 5, Category 1, Category 3, Category 2]...
Path: .
x: TextList (2055 items)
[Text xxbos i think need xxup emm xxup exe noems xxmaj this let expanded mem active use therefore give extended mem windows yet bases covered i use dos i familiar c ya artin, Text xxbos xxmaj sorry olwm tvtwm xxmaj they place title position window position xxmaj this becomes problem want program able save current configuration restore later xxmaj currently solution xcreatewindow xxunk xxunk xflush xxunk xxmaj it works olwm less xxunk xxmaj all leaves wondering i xxunk something obvious xxmaj richard i know mwm resource specify positions used border window user window xxmaj maybe window managers similar options xxmaj another way figure difference user window position window manager decoration window position subtract positions xxmaj you use xquerytree remember window manager xxunk window parent window xxmaj unfortunately figur

In [0]:
# train the learner object with learning rate = 1e-2
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,5.619023,4.919724,0.242994


In [0]:
learn.save_encoder('ft_enc')

In [0]:
learn = text_classifier_learner(data_clas, drop_mult=0.7)
learn.load_encoder('ft_enc')

In [0]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,1.570720,1.489627,0.382520


In [0]:
# get predictions
preds, targets = learn.get_preds()

predictions = np.argmax(preds, axis = 1)
pd.crosstab(predictions, targets)

col_0,0,1,2,3,4
row_0,,,,,
0,6,3,2,2,5
1,24,44,13,9,28
2,12,29,56,23,6
3,66,75,97,126,34
4,67,26,9,14,105


In [0]:
# {0: 4, 1: 1, 2: 2, 3: 5, 4: 3}
 # 1=comp.graphic, 2= comp.os.ms-windows.misc, 3 = comp.sys.ibm.pc.hardware, 4 = comp.sys.mac.hardware, 5=comp.windows.x, 


In [0]:
table = pd.crosstab(predictions, targets)
for i in table:
  index_to_label = dic
  table.targets[i] = dic[]
  
  

In [0]:
learn.get_preds()

[tensor([[0.1220, 0.1047, 0.5156, 0.1435, 0.1141],
         [0.1395, 0.1604, 0.3910, 0.1552, 0.1540],
         [0.1441, 0.1445, 0.4326, 0.1450, 0.1339],
         ...,
         [0.1736, 0.0745, 0.0351, 0.3823, 0.3345],
         [0.1736, 0.0745, 0.0351, 0.3823, 0.3345],
         [0.1736, 0.0745, 0.0351, 0.3823, 0.3345]]),
 tensor([1, 1, 1, 1, 4, 1, 4, 4, 2, 0, 4, 1, 4, 0, 0, 4, 4, 2, 0, 1, 3, 2, 3, 1,
         2, 3, 0, 0, 0, 0, 2, 1, 0, 1, 4, 2, 2, 4, 1, 3, 2, 3, 1, 3, 2, 4, 2, 1,
         0, 3, 1, 4, 2, 2, 0, 4, 4, 4, 4, 3, 4, 2, 2, 1, 4, 4, 0, 1, 1, 4, 4, 4,
         1, 2, 4, 0, 4, 3, 0, 0, 0, 4, 2, 3, 0, 4, 4, 2, 2, 4, 3, 3, 1, 0, 1, 2,
         2, 1, 1, 2, 4, 1, 4, 3, 3, 0, 0, 4, 1, 2, 4, 4, 3, 0, 4, 1, 3, 2, 3, 2,
         4, 1, 3, 1, 1, 4, 4, 1, 0, 2, 1, 0, 0, 4, 4, 2, 4, 1, 1, 2, 2, 3, 3, 2,
         3, 2, 3, 4, 4, 3, 1, 0, 3, 4, 2, 0, 4, 0, 0, 2, 4, 0, 3, 4, 0, 0, 4, 3,
         4, 3, 1, 2, 3, 4, 3, 3, 0, 4, 2, 3, 0, 2, 1, 2, 4, 2, 0, 0, 1, 0, 0, 3,
         0, 3, 3, 0, 1, 1, 4, 

In [0]:
predictions

tensor([2, 2, 2, 2, 4, 2, 4, 4, 2, 2, 4, 2, 2, 4, 4, 4, 4, 2, 0, 1, 2, 2, 0, 0,
        2, 3, 0, 4, 4, 0, 0, 1, 2, 2, 4, 3, 2, 4, 2, 3, 2, 2, 3, 1, 2, 4, 2, 2,
        4, 3, 1, 4, 2, 1, 3, 4, 0, 4, 4, 2, 0, 2, 2, 3, 4, 1, 3, 1, 1, 4, 4, 1,
        1, 3, 4, 1, 4, 2, 4, 1, 0, 4, 1, 3, 4, 1, 0, 3, 2, 1, 3, 3, 3, 1, 2, 3,
        2, 2, 1, 1, 4, 1, 4, 3, 3, 2, 4, 4, 2, 1, 4, 1, 3, 0, 1, 1, 1, 3, 1, 3,
        4, 1, 2, 4, 2, 4, 4, 3, 3, 2, 2, 1, 1, 1, 1, 3, 2, 1, 2, 2, 2, 3, 2, 3,
        3, 2, 3, 4, 1, 3, 1, 4, 3, 4, 2, 4, 3, 4, 1, 3, 2, 4, 3, 1, 2, 1, 4, 3,
        4, 3, 1, 3, 2, 4, 2, 3, 3, 4, 3, 1, 4, 0, 1, 2, 3, 3, 3, 1, 1, 4, 3, 3,
        3, 3, 3, 4, 3, 1, 1, 3, 1, 4, 4, 3, 4, 2, 3, 4, 1, 1, 3, 2, 3, 3, 2, 3,
        2, 0, 4, 1, 3, 1, 2, 3, 3, 3, 1, 4, 3, 2, 1, 2, 1, 4, 1, 3, 3, 3, 1, 4,
        4, 3, 3, 2, 3, 3, 2, 3, 2, 1, 1, 3, 4, 3, 3, 4, 3, 1, 3, 3, 3, 3, 2, 1,
        3, 2, 1, 2, 4, 2, 3, 2, 3, 1, 3, 2, 2, 3, 4, 3, 2, 3, 3, 3, 2, 1, 2, 3,
        2, 1, 3, 3, 4, 1, 1, 3, 3, 2, 1,

In [0]:
targets

tensor([1, 1, 1, 1, 4, 1, 4, 4, 2, 0, 4, 1, 4, 0, 0, 4, 4, 2, 0, 1, 3, 2, 3, 1,
        2, 3, 0, 0, 0, 0, 2, 1, 0, 1, 4, 2, 2, 4, 1, 3, 2, 3, 1, 3, 2, 4, 2, 1,
        0, 3, 1, 4, 2, 2, 0, 4, 4, 4, 4, 3, 4, 2, 2, 1, 4, 4, 0, 1, 1, 4, 4, 4,
        1, 2, 4, 0, 4, 3, 0, 0, 0, 4, 2, 3, 0, 4, 4, 2, 2, 4, 3, 3, 1, 0, 1, 2,
        2, 1, 1, 2, 4, 1, 4, 3, 3, 0, 0, 4, 1, 2, 4, 4, 3, 0, 4, 1, 3, 2, 3, 2,
        4, 1, 3, 1, 1, 4, 4, 1, 0, 2, 1, 0, 0, 4, 4, 2, 4, 1, 1, 2, 2, 3, 3, 2,
        3, 2, 3, 4, 4, 3, 1, 0, 3, 4, 2, 0, 4, 0, 0, 2, 4, 0, 3, 4, 0, 0, 4, 3,
        4, 3, 1, 2, 3, 4, 3, 3, 0, 4, 2, 3, 0, 2, 1, 2, 4, 2, 0, 0, 1, 0, 0, 3,
        0, 3, 3, 0, 1, 1, 4, 3, 1, 4, 1, 3, 4, 3, 3, 4, 2, 1, 2, 2, 0, 1, 1, 3,
        2, 3, 0, 1, 2, 4, 2, 3, 2, 1, 1, 4, 4, 1, 4, 2, 4, 4, 1, 2, 2, 0, 1, 4,
        1, 3, 0, 2, 2, 2, 1, 3, 2, 1, 2, 1, 0, 2, 3, 0, 3, 0, 3, 2, 2, 3, 0, 4,
        3, 2, 1, 2, 4, 2, 2, 1, 3, 4, 0, 2, 2, 1, 4, 2, 2, 3, 3, 3, 3, 4, 2, 1,
        2, 0, 3, 3, 4, 1, 2, 3, 4, 2, 0,

In [0]:
df_val.head()

,label,text
index,,
1476,4,When I owned SE I replaced fan SE Silencer ava...
1621,1,I given sites excellent D objects sorts file f...
415,4,Is anyone using greyscale handscanner I thinki...
2230,2,MAX AX AX AX AX AX AX AX AX AX AX AX AX AX AX ...
62,1,The file contents listings Knowledge Media Res...


In [0]:
df_val.sort_index(ascending=True)  # df.sort_index(inplace=True)

,label,text
index,,
2,4,The title says I need know c rom versions
4,4,Terry hi I recently bought LCIII Datadesk E I ...
6,4,terminated It possible connect another interna...
9,5,Reminder Plan Andrew Conference Date The dates...
12,2,I reading much confusion whether use ATManager...
14,5,return place probably incorrect It pointer int...
17,5,Try char name NULL unsigned long value XGetFon...
18,4,None You need buy ns k VRAM SIMMs They cost fa...
20,2,


In [0]:
#pd.crosstab(predictions, targets) columns=variable2
table = pd.crosstab(predictions, targets)
result = table.reindex(index=predictions, columns=targets, fill_value=0)
result

In [0]:
print(table)

col_0    0   1    2    3    4
row_0                        
0       38  11    9    9   13
1       15  29   10   12   21
2       52  99  158  123   48
3        7  11   13   26    2
4      122  87   46   61  153


In [0]:
df_val.head()

,label,text
1476,4,When I owned SE I replaced fan SE Silencer ava...
1621,1,I given sites excellent D objects sorts file f...
415,4,Is anyone using greyscale handscanner I thinki...
2230,2,MAX AX AX AX AX AX AX AX AX AX AX AX AX AX AX ...
62,1,The file contents listings Knowledge Media Res...


In [0]:
df_val['label'].value_counts()

5    178
3    177
2    177
1    175
4    174
Name: label, dtype: int64

In [0]:
df2 = df[df['label'].isin([1, 2, 3, 4, 5])]  # 1=comp.graphic, 2= comp.os.ms-windows.misc, 3 = comp.sys.ibm.pc.hardware, 4 = comp.sys.mac.hardware, 5=comp.windows.x, 

In [0]:

CLASSES = ['neg', 'pos', 'unsup']

def get_texts(path):
    texts,labels = [],[]
    for idx,label in enumerate(CLASSES):
        for fname in (path/label).glob('*.*'):
            texts.append(fname.open('r', encoding='utf-8').read())
            labels.append(idx)
    return np.array(texts),np.array(labels)

trn_texts,trn_labels = get_texts(PATH/'train')
val_texts,val_labels = get_texts(PATH/'test')

In [0]:
dic = dict (enumerate(df2.label.unique()))
dic

# {0: 4, 1: 1, 2: 2, 3: 5, 4: 3}
 # 1=comp.graphic, 2= comp.os.ms-windows.misc, 3 = comp.sys.ibm.pc.hardware, 4 = comp.sys.mac.hardware, 5=comp.windows.x, 

{0: 4, 1: 1, 2: 2, 3: 5, 4: 3}

In [0]:
df3 = df[df['label'].isin([3,4])]  #  1 = comp.graphic, 10 = rec.sport.hockey
df3 = df3.reset_index(drop = True)

In [0]:
df3['label'].value_counts()

3    590
4    578
Name: label, dtype: int64

In [0]:
#Data Preprocessing 
df3['text'] = df3['text'].str.replace("[^a-zA-Z]", " ")

In [0]:
# tokenization 
tokenized_doc = df3['text'].apply(lambda x: x.split())

# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# de-tokenization 
detokenized_doc = [] 
for i in range(len(df3)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

df3['text'] = detokenized_doc

In [0]:
from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn, df_val = train_test_split(df3, stratify = df3['label'], test_size = 0.3, random_state = 12)

In [0]:
df_trn.shape, df_val.shape

((817, 2), (351, 2))

In [0]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

In [0]:
#Fine-Tuning 
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103, drop_mult=0.7)

In [0]:
# train the learner object with learning rate = 1e-2
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,7.630622,6.603959,0.088823


In [0]:
learn.save_encoder('ft_enc')

In [0]:
learn = text_classifier_learner(data_clas, drop_mult=0.7)
learn.load_encoder('ft_enc')

In [0]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.705888,0.679510,0.584046


In [0]:
# get predictions
preds, targets = learn.get_preds()

predictions = np.argmax(preds, axis = 1)
pd.crosstab(predictions, targets)

col_0,0,1
row_0,,
0,71,40
1,106,134


In [0]:
# {0: 4, 1: 1, 2: 2, 3: 5, 4: 3}
 # 1=comp.graphic, 2= comp.os.ms-windows.misc, 3 = comp.sys.ibm.pc.hardware, 4 = comp.sys.mac.hardware, 5=comp.windows.x, 
  
  dic = dict (enumerate(df3.label.unique()))
dic

{0: 4, 1: 3}

In [0]:
df_trn[:5].sort_index()

,label,text
162,4,I Mac IIsi considering upgrades cards hard dri...
606,3,From article qq INNdqc dns NMSU Edu bgrubb dan...
983,3,While others may better experiences I share se...
991,3,Hi When posting Winmark results good idea give...
1166,4,I would like know people opinions real world d...


In [0]:
df3.label.unique()

array([4, 3])